In [1]:
%cd "/content/"

/content


In [2]:
!git clone https://github.com/USC-MCL/EE569_2020Spring.git

Cloning into 'EE569_2020Spring'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 133 (delta 78), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (133/133), 37.10 KiB | 584.00 KiB/s, done.
Resolving deltas: 100% (78/78), done.


In [3]:
!pwd
!ls EE569_2020Spring/

/content
cross_entropy.py  lag.py   pixelhop2.py  requirements.txt
cwSaab.py	  llsr.py  README.md	 saab.py


In [4]:
!pwd

/content


In [5]:
%cd /content/EE569_2020Spring/

/content/EE569_2020Spring


In [0]:
import numpy as np
from cross_entropy import Cross_Entropy
from lag import LAG
from llsr import LLSR as myLLSR
from pixelhop2 import Pixelhop2
import skimage
from skimage.util import view_as_windows
import skimage.measure
import numpy as np

In [0]:
def Concat(X, concatArg):
    return X

In [0]:
def Shrink(X, shrinkArg):
    # print(type(shrinkArg))

    # win = shrinkArg['win']
    win = 5
    hop1 = shrinkArg['hop']
    # stride = shrinkArg['stride']

    # lasthop = shrinkArgs['hoplast']
    lasthop = False
    stride = 1
    ch = X.shape[-1]
    print('input shape:',X.shape)
    if lasthop:
      # X = view_as_windows(X, (1,win,win,1), (1,win,win,1))
      X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
    else:
      if ~hop1:
        # win1=2
        X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
      X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
      X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
      # if hop1:
        # X = skimage.measure.block_reduce(X, (1,win,win,1), np.max)
      print('output shape:',X.shape)
    return X  

In [0]:
# from skimage.util import view_as_windows
# def Shrink(X, shrinkArg):

#   win = shrinkArg['win']
#   win=5
#   hop1 = shrinkArg['hop']
#   # print(shrinkArg['hop'])
#   stride = 1
#   ch = X.shape[-1]
#   # if ~hop1:
#   #   X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
#     # keras.layers.MaxPooling2D(pool_size=(2, 2), strides=1, padding='same', data_format=None)
#   X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
#   X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
#   # print(X.shape)
#   # if hop1:
#   #   X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
#   return X

In [0]:
from skimage.util import view_as_windows
def Shrink(X, shrinkArg):

  win = shrinkArg['win']
  win=5
  hop = shrinkArg['hop']
  print('Hop:',hop)
  stride = 1
  ch = X.shape[-1]
  print('Input shape:',X.shape)
  if hop==1:
    X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
    X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
  if hop==2:
    X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
    X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
  if hop==3:
    X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
  print(X.shape)
  return X

In [0]:

"""neighborhood construction
shrink saab concat args
"""
SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'useDC':True, 'batch':None, 'cw': True},
                {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw': True}]
shrinkArgs = [{'func':Shrink, 'win':2, 'hop':1},
              {'func': Shrink, 'win':2,'hop':2},
              {'func': Shrink, 'win':2, 'hop':3}]
concatArg = {'func':Concat}

In [12]:
from keras.datasets import cifar10
#preprocessing of data 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train_50k = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

Using TensorFlow backend.


170500096/170498071 [==============================] - 6s 0us/step


In [13]:
#selecting 10k images
x_train_10k = x_train_50k[0:1000]
print(x_train_10k.shape)

(1000, 32, 32, 3)


In [14]:
#train 
phops = Pixelhop2(depth=3, TH1=0.01, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
phops.fit(x_train_10k)
output = phops.transform(x_train_10k)

pixelhop2 fit
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 

In [15]:
  print(output[0].shape)
  print(output[1].shape)
  print(output[2].shape)

(1000, 14, 14, 45)
(1000, 5, 5, 147)
(1000, 1, 1, 219)


In [16]:
#save model
#train 
phops = Pixelhop2(depth=3, TH1=0.01, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
phops.fit(x_train_10k)

import pickle

# save
with open('pixelhop2.pkl','wb') as f:
    pickle.dump(phops,f)

# load
with open('pixelhop2.pkl', 'rb') as f:
    clf2 = pickle.load(f)


pixelhop2 fit
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 

In [17]:
#all train data - 50k images
#save model
#train 
phops = Pixelhop2(depth=3, TH1=0.01, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
phops.fit(x_train_50k)
output = phops.transform(x_train_50k)

pixelhop2 fit
Hop: 1
Input shape: (50000, 32, 32, 1)
(50000, 14, 14, 25)
Hop: 1
Input shape: (50000, 32, 32, 1)
(50000, 14, 14, 25)
Hop: 1
Input shape: (50000, 32, 32, 1)
(50000, 14, 14, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Input shape: (50000, 14, 14, 1)
(50000, 5, 5, 25)
Hop: 2
Inpu

In [0]:
def cal_cross_entropy(features,Ns,y_train):
  ce = Cross_Entropy(num_class=10, num_bin=5)
  features = features.reshape((features.shape[0], -1))
  print(features.shape)
  feat_ce = np.zeros((features.shape[0],features.shape[-1]))
  for l in range(features.shape[0]):
    for k in range(features.shape[-1]):
      feat_ce[l,k] = ce.KMeans_Cross_Entropy(features[l,k].reshape(-1,1), y_train)
      # print(" --> KMeans ce: %s"%str(feat_ce[k]))
      # print("------- DONE -------\n")
  # print(feat_ce.shape)
  # ce_sorted = feat_ce.sort()
  # ce_ns = ce_sorted[0:Ns]
  return feat_ce

In [19]:
print(output[0].shape[-1])
k = output[0].reshape((output[0].shape[0], -1))
# print(type(int(k)))
print(k.shape)

45
(50000, 8820)


In [0]:
ce_hop_unit1 = cal_cross_entropy(output[0],4000,y_train)
ce_hop_unit2 = cal_cross_entropy(output[1],4000,y_train)
ce_hop_unit3 = cal_cross_entropy(output[2],4000,y_train)

(50000, 8820)


In [0]:
a = np.asarray([[1,2],[3,4],[2,2]])
print(a.shape)
a = np.mean(a, axis=0)
print(a)

In [0]:
def crossentropy_sort(crossentropy,Ns):
  crossentropy = np.expand_dims(crossentropy,axis=0)
  print(crossentropy.shape)
  ce_avg = np.mean(crossentropy,axis=0)
  print(ce_avg)
  ce_sorted = sorted(ce_avg)
  ce_final = np.zeros(crossentropy.shape)
  for i in range(Ns):
    indx[i] = np.argwhere(ce_avg==ce_sorted)
    ce_final[:,i] = ce_avg(indx[i])
  #reduce dimensions selecting Ns dimensions
  # print(len(ce_sorted))
  # ce_ns = ce_sorted[0:Ns]
  return ce_final

In [0]:
# print(ce_hop_unit1.shape)
# ce_sorted = sorted(ce_hop_unit1)
# print(len(ce_sorted))
# ce_ns = ce_sorted[0:4000]

In [0]:
Ns = 25000
ce_sorted_unit1 = crossentropy_sort(ce_hop_unit1,Ns)
ce_sorted_unit2 = crossentropy_sort(ce_hop_unit2,Ns)
ce_sorted_unit3 = crossentropy_sort(ce_hop_unit3,Ns)
print(len(ce_sorted_unit1))

In [0]:
def lag_compute(x_train, y_train,x_test, y_test,alpha=10):
    x_train = np.asarray(x_train)
    x_train = np.expand_dims(x_train, axis=1)
    print(x_train.shape)
    lag = LAG(encode='distance', num_clusters=[2,2,2,2,2,2,2,2,2,2], alpha=10, learner=myLLSR(onehot=False))  
    lag.fit(x_train, y_train)
    x_train_trans = lag.transform(x_train)
    x_train_predprob = lag.predict_proba(x_train)
    print(" --> train acc: %s"%str(lag.score(x_train, y_train)))
    print(" --> test acc.: %s"%str(lag.score(x_test, y_test)))
    print("------- DONE -------\n")
    return x_train_trans

In [0]:
alpha = 10
lag_unit1 = lag_compute(ce_sorted_unit1,y_train,x_test, y_test,alpha)
lag_unit2 = lag_compute(ce_sorted_unit2,y_train,x_test, y_test,alpha)
lag_unit3 = lag_compute(ce_sorted_unit3,y_train,x_test, y_test,alpha)

In [0]:
#concatenate the outputs of lag
lags = np.concatenate((lag_unit1,lag_unit2,lag_unit3),axis = 3)

In [0]:
#classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(lags, y_train)
RandomForestClassifier(max_depth=2, random_state=0)
print(clf.feature_importances_)
print(clf.predict(x_test))


In [0]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import itertools
plt.rcParams['figure.figsize'] = [10,7]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()


p_test = clf.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [0]:
import numpy as np
a = np.ones((3,4,5))
b = np.ones((3,4,5))
c = np.concatenate((a[...,np.newaxis],b[...,np.newaxis]),axis=3)

In [0]:
#feature selection

from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
    
print(" > This is a test example: ")
digits = datasets.load_digits()
X = digits.images.reshape((len(digits.images), -1))
print(" input feature shape: %s"%str(X.shape))
X_train, X_test, y_train, y_test = train_test_split(X, digits.target, test_size=0.2,  stratify=digits.target)
    
ce = Cross_Entropy(num_class=10, num_bin=5)
feat_ce = np.zeros(X_train.shape[-1])
for k in range(X_train.shape[-1]):
    feat_ce[k] = ce.KMeans_Cross_Entropy(X_train[:,k].reshape(-1,1), y_train)
    print(y_train)
    print(" --> KMeans ce: %s"%str(feat_ce[k]))
print("------- DONE -------\n")
print(feat_ce.shape)

In [0]:
digits = datasets.load_digits()
X = digits.images.reshape((len(digits.images), 8, 8, 1))
print(X.shape)

In [0]:
# example useage
    from sklearn import datasets
    from skimage.util import view_as_windows

    # example callback function for collecting patches and its inverse
    def Shrink(X, shrinkArg):
        win = shrinkArg['win']
        X = view_as_windows(X, (1,win,win,1), (1,win,win,1))
        return X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)

    # example callback function for how to concate features from different hops
    def Concat(X, concatArg):
        return X

    # read data
    import cv2
    print(" > This is a test example: ")
    # digits = datasets.load_digits()
    # X = digits.images.reshape((len(digits.images), 8, 8, 1))
    # print(" input feature shape: %s"%str(X.shape))
    from keras.datasets import cifar10
#preprocessing of data 
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train_50k = x_train.astype('float32')/255
    x_test = x_test.astype('float32')/255 
    X = x_train_50k[0:1000]
    # set args
    SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'useDC':True, 'batch':None, 'cw':False}, 
                {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw':True},
                {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw':True}]
    shrinkArgs = [{'func':Shrink, 'win':2}, 
                  {'func': Shrink, 'win':2},
                {'func': Shrink, 'win':2}]
    concatArg = {'func':Concat}

    print(" --> test inv")
    print(" -----> depth=1")
    p2 = Pixelhop2(depth=1, TH1=0.005, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
    p2.fit(X)
    output = p2.transform(X)
    print(" -----> depth=2")
    p2 = Pixelhop2(depth=3, TH1=0.005, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
    p2.fit(X)
    output = p2.transform(X)
    print(output[0].shape, output[1].shape,output[2].shape)
    print("------- DONE -------\n")
